
## Import dependencies

In [1]:
import os
import time

import pandas
import torch

from proselflc.trainer.trainer_cnn_vision_derivedgrad import Trainer

import pprint
print = pprint.PrettyPrinter(indent=4).pprint

## Add params configurations

In [2]:
params = {}
params.update(
    {
        "data_name": "cifar100",
        "num_classes": 100,  # 1000
        "device": "gpu",
        #
        "num_workers": 8,
        #
        "counter": "iteration",
        "classes_per_batch": 8,
        #
        # To get deterministic results, please uncomment the line below
        # to set seed parameter.
        # "seed": 123,
    }
)

# data
params["symmetric_noise_rate"] = 0.4

# network
params["network_name"] = "shufflenetv2"

# for the demo purpose, I set the total epochs to be small.
params["total_epochs"] = 100
params["eval_interval"] = 500 # iterations

# batch
params["batch_size"] = 128
params["sampler"] = "BalancedBatchSampler"
params["classes_per_batch"] = 64

# learning rate
params["lr"] = 0.2
params["weight_decay"] = 1e-3
params["lr_scheduler"] = "WarmupMultiStepSchedule"
params["warmup_epochs"] = 0
params["milestones"] = [20000, 30000]

# optimisation
params["momentum"] = 0.9
params["batch_accumu_steps"] = 10
params["gamma"] = 0.1

# loss settings
params["loss_mode"] = "cross entropy"
params["trust_mode"] = "global*(1-H(p)/H(u))"
params["loss_name"] = "proselflc"
params["transit_time_ratio"] = 0.50
params["exp_base"] = 16
params["logit_soften_T"] = 0.5

print(params)

{   'batch_accumu_steps': 10,
    'batch_size': 128,
    'classes_per_batch': 64,
    'counter': 'iteration',
    'data_name': 'cifar100',
    'device': 'gpu',
    'eval_interval': 500,
    'exp_base': 16,
    'gamma': 0.1,
    'logit_soften_T': 0.5,
    'loss_mode': 'cross entropy',
    'loss_name': 'proselflc',
    'lr': 0.2,
    'lr_scheduler': 'WarmupMultiStepSchedule',
    'milestones': [20000, 30000],
    'momentum': 0.9,
    'network_name': 'shufflenetv2',
    'num_classes': 100,
    'num_workers': 8,
    'sampler': 'BalancedBatchSampler',
    'symmetric_noise_rate': 0.4,
    'total_epochs': 100,
    'transit_time_ratio': 0.5,
    'trust_mode': 'global*(1-H(p)/H(u))',
    'warmup_epochs': 0,
    'weight_decay': 0.001}


## Extra configurations to get deterministic results

In [3]:
%env CUDA_VISIBLE_DEVICES=0
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
params["seed"] = 123

env: CUDA_VISIBLE_DEVICES=0
env: CUBLAS_WORKSPACE_CONFIG=:4096:8


## Create the folder to store intermediate and final results
* First, run `sudo mkdir /home/proselflc_experiments/ && sudo chmod -R 777 /home/proselflc_experiments/` so that so you have the write permission.
* Or set `WORK_DIR = "/home/your_username/proselflc_experiments/"`

In [4]:
WORK_DIR = "/home/proselflc_experiments/"
#
# use the time as a unique experiment identifier
dt_string = time.strftime("%Y%m%d-%H%M%S")
summary_writer_dir = (
    params["loss_name"]
    + "_"
    + dt_string
)
params["summary_writer_dir"] = (
    WORK_DIR
    + "/"
    + params["data_name"]
    + "_symmetric_noise_rate_"
    + str(params["symmetric_noise_rate"])
    + "/"
    + params["network_name"]
    + "/"
    + summary_writer_dir
)
if not os.path.exists(params["summary_writer_dir"]):
    os.makedirs(params["summary_writer_dir"])

## Init the trainer, and store the params configurations
For each experiment, we have one unique result folder to store the params configurations and learning curves.
Therefore, you can revisit any specific experiment whenever you need without losing any details.

In [5]:
trainer = Trainer(params=params)
# params["milestones"] was a list of integers, we convert it to a string before sinking.
params["milestones"] = str(params["milestones"])
dataframe = pandas.DataFrame(params, index=[0])
dataframe.to_csv(
    params["summary_writer_dir"] + "/params.csv",
    encoding="utf-8",
    index=False,
    sep="\t",
    mode="w",  #
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Run the trainer and save the final model

In [6]:
trainer.train()
torch.save(
    trainer.network,
    params["summary_writer_dir"] + "/model.pt",
)
print(
    "The experiment is finished with details sinked in {}".format(
        params["summary_writer_dir"]
    )
)

Evaluating Network.....
Iteration= (500,2)/(39000, 100), lr=0.2000, batch_mean_epsilon=0.0001, valid batch size=128.0000, batch_mean_gtrust=0.0004, batch_mean_etrust=0.1431
clean_test: Loss= 4.2136, Accuracy= 0.0633, Entropy= 0.9512, Max_p= 0.0415,
noisy_subset: Loss= 4.7869, Accuracy= 0.0092, Entropy= 0.9583, Max_p= 0.0382,
clean_subset: Loss= 4.2297, Accuracy= 0.0606, Entropy= 0.9580, Max_p= 0.0383,
cleaned_noisy_subset: Loss= 4.2430, Accuracy= 0.0582, Entropy= 0.9583, Max_p= 0.0382,
Evaluating Network.....
Iteration= (1000,3)/(39000, 100), lr=0.2000, batch_mean_epsilon=0.0001, valid batch size=128.0000, batch_mean_gtrust=0.0005, batch_mean_etrust=0.1968
clean_test: Loss= 3.9628, Accuracy= 0.1056, Entropy= 0.9309, Max_p= 0.0635,
noisy_subset: Loss= 4.8374, Accuracy= 0.0106, Entropy= 0.9379, Max_p= 0.0574,
clean_subset: Loss= 3.9745, Accuracy= 0.1059, Entropy= 0.9372, Max_p= 0.0580,
cleaned_noisy_subset: Loss= 4.0047, Accuracy= 0.0975, Entropy= 0.9379, Max_p= 0.0574,
Evaluating Networ

KeyboardInterrupt: 